In [1]:
'This is my gensim with LDA'
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pandas as pd
import spacy
import numpy as np
from nltk.corpus import stopwords

import pyLDAvis
import pyLDAvis.gensim



In [2]:
#import my data
df = pd.read_excel("Mentor_CSAT.xlsx")
df.head()

,Unnamed: 0,activityid,Lastdate,SentDate,StudentName,PMName,PMOperationsID,PathID,PipePathwayID,StudentType,...,"To help us better understand your experience, please explain the reasons for your satisfaction ratin",My Mentor shares useful information.,My Mentor responds to me within 24 hours.,My Mentor helps me get questions and issues resolved.,My Mentor cares about me and my success.,Sent Date,Semester,% Communicated,% Not Communicated,"Overall, how satisfied are you with BYU-Pathway Mentoring?"
0,0,F22B8E8C-476D-EE11-8DF0-00224808045A,2023-10-17 23:47:33,10/16/2023,Hnin Wai Oo,Maria Carmela Villenas Gaela,10260,22822379,22822379,Matriculation,...,She's really kind to me,Mostly true,Moderately true,Slightly true,Moderately true,2023-10-16,Fall 2023,Communicated,NaN,Satisfied
1,1,C049BF3C-0F10-EE11-8F6D-0022480A40EB,2023-06-21 08:40:19,6/20/2023,Adrian James Oriewo,Susan Kapiya Chitembo,10121,22560197,22560197,Online Degree - BYU-Idaho,...,The Peer Mentor always encourages me to keep o...,Definitely true,Definitely true,Definitely true,Definitely true,2023-06-20,Spring 2023,Communicated,NaN,Very satisfied
2,2,15D312C2-80F9-ED11-8849-0022480AE0F6,2023-05-23 15:44:56,5/22/2023,Marie Angelene Empleo Diaz,Julie Anne Gollayan Alberto Espino,10503,22525995,22525995,PathwayConnect,...,She replies whenever I need her help.,Moderately true,Moderately true,Moderately true,Moderately true,2023-05-22,Spring 2023,Communicated,NaN,Very satisfied
3,3,8F082B81-0C25-EE11-9CBE-0022480AE0F6,2023-07-18 01:43:35,7/17/2023,Gabart Eugenio Jean Louis,Michael Aboagye Adu,10389,20062864,20062864,PathwayConnect,...,Best mentor ever thus far,Definitely true,Definitely true,Definitely true,Definitely true,2023-07-17,Spring 2023,Communicated,NaN,Very satisfied
4,4,264787A9-EC98-EE11-BE37-0022480AE1C8,2023-12-12 12:47:48,12/11/2023,Nomathamsanqa Maho,Evans Nana Baah Ankomah,10243,13242085,13242085,Online Degree - Ensign College,...,NaN,NaN,NaN,NaN,NaN,2023-12-11,Fall 2023,NaN,Not Communicated,NaN


In [3]:
stopwords = stopwords.words('english')

In [4]:
data = df['To help us better understand your experience, please explain the reasons for your satisfaction ratin'].values



In [5]:
def lemmatization(texts, allowed_posttags = ["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(str(text))
        new_text = []
        for token in doc:
            if token.pos_ in allowed_posttags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        if final != '':
            texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print(lemmatized_texts[0:5])

['really kind', 'always encourage keep push foward study most importantly share scripture strengthen', 'reply need help', 'good mentor ever thus far', 'very satisfied peer mentor always available need always answer question very important']


In [6]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text,deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)
print(data_words[0:5])

[['really', 'kind'], ['always', 'encourage', 'keep', 'push', 'foward', 'study', 'most', 'importantly', 'share', 'scripture', 'strengthen'], ['reply', 'need', 'help'], ['good', 'mentor', 'ever', 'thus', 'far'], ['very', 'satisfied', 'peer', 'mentor', 'always', 'available', 'need', 'always', 'answer', 'question', 'very', 'important']]


In [7]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print(corpus[0:5])
word = id2word[[0][0]]
print(word)

[[(0, 1), (1, 1)], [(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)], [(13, 1), (14, 1), (15, 1)], [(16, 1), (17, 1), (18, 1), (19, 1), (20, 1)], [(2, 2), (14, 1), (19, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 2)]]
kind


In [8]:
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus, id2word=id2word, num_topics= 10, random_state=100,update_every=1,chunksize=100,passes=10, alpha="auto")

In [9]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

c:\Python311\Lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Python311\Lib\site-packages\sklearn\manifold\_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.397891  0.263347       1        1  29.580286
3      0.118147  0.441225       2        1  14.205938
1     -0.359512 -0.266330       3        1  12.987170
9     -0.257389  0.413634       4        1   9.632162
8      0.199274 -0.435606       5        1   7.747045
0     -0.155232  0.149822       6        1   6.180649
4     -0.107315 -0.406509       7        1   5.980963
2     -0.465613  0.029212       8        1   5.680377
6      0.179483 -0.080237       9        1   5.288211
7      0.450264 -0.108557      10        1   2.717200, topic_info=         Term          Freq         Total Category  logprob  loglift
19     mentor  10164.000000  10164.000000  Default  30.0000  30.0000
13       help  14518.000000  14518.000000  Default  29.0000  29.0000
27       very   5996.000000   5996.000000  Default  28.0000  28.0000
2      always  12956.000000  12956.000000  Default  27.0000  27.0000
24       peer   5189.000000   5189.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
1308  barrier     66.291543     67.172346  Topic10  -4.9177   3.5924
1394    allow     63.902347     64.783223  Topic10  -4.9544   3.5919
681   prepare     63.358646     64.239453  Topic10  -4.9629   3.5918
587    result     57.473614     58.354339  Topic10  -5.0604   3.5904
992   mindset     52.219638     53.100419  Topic10  -5.1563   3.5888

[330 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
191       9  0.998849       able
640       5  0.999127   academic
716       9  0.992687     access
255       5  0.995903    achieve
251       7  0.998294     active
...     ...       ...        ...
225       5  0.997340  wonderful
60        2  0.998658       word
171       3  0.998768       work
1164      8  0.996102       year
611       7  0.997710        yet

[300 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 4, 2, 10, 9, 1, 5, 3, 7, 8])